In [55]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.llms import Ollama
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_experimental.llms.ollama_functions import OllamaFunctions

In [56]:
# Configuração p/ modelo local
model_name = "mario:latest"
host = "http://localhost:8081"

In [57]:
# Inicialização do modelo com set de temperatura
llm = Ollama(model=model_name, base_url=host, temperature=0.1)

In [58]:
# Extração de conteúdo da página web
url = "https://g1.globo.com/sp/sorocaba-jundiai/noticia/2024/02/06/prefeitura-de-jundiai-identifica-mais-um-morcego-contaminado-com-raiva-cidade-soma-cinco-casos-no-ano.ghtml"
loader = WebBaseLoader(url)
docs = loader.load()

In [59]:
embeddings = OllamaEmbeddings(model=model_name, base_url=host)

In [60]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [65]:
# Criação do modelo de prompt.
prompt_template = """Answer the following question based only on the context:
{context}

Question: {input}
"""

prompt = ChatPromptTemplate.from_template(prompt_template)

document_chain = create_stuff_documents_chain(llm, prompt)

In [66]:
# Inicialização do retriver (escolhe a melhor fonte de dados p/ gerar a resposta)
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [67]:
# Interação com o documento
response = retrieval_chain.invoke({"input": "What is the article about?"})

print(response["answer"].replace('<|endoftext|>',''))

The article is about a bat infected with rabies being found in Jundiaí, Brazil. The city has had five cases of rabies this year, and the bat was found on a sidewalk in the Vila Nova neighborhood. The city's health department recommends vaccinating dogs and cats against the disease and advises residents on how to handle contact with infected animals.


In [72]:
# Interação com o documento
response = retrieval_chain.invoke({"input": "How the bat was found?"})

print(response["answer"].replace('<|endoftext|>',''))

The bat was found in a sidewalk in the city of Jundiaí, in the state of São Paulo.


In [73]:
# Interação com o documento
response = retrieval_chain.invoke({"input": "There is some medicine to take in case of infection?"})

print(response["answer"].replace('<|endoftext|>',''))

Yes, there are vaccines available for the prevention of rabies. It is recommended that people who have been bitten or scratched by an animal seek medical attention and get vaccinated as soon as possible.


In [75]:
# Interação com o documento
response = retrieval_chain.invoke({"input": "Which are the reccomendations?"})

print(response["answer"].replace('<|endoftext|>',''))

The recommendations are to vaccinate dogs and cats against rabies, avoid contact with wild animals such as bats, and seek medical attention if bitten or scratched by a bat.
